# Transformers

modified from github repo linked to from https://peterbloem.nl/blog/transformers



In [11]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.distributions as dist
import time
import random, math, sys, tqdm, math, gzip

In [12]:
######################################################################
# from util.py

def d(tensor=None):
    # return torch.device('cpu')
    return torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

def slice_diag(matrix, l, dv=None):
    """
    Take a batch of attention matrices for relative position encodings and slice out the relevant attentions. These
    are the length l sequences starting at the diagonal
    """
    if dv is None:
        dv = d(matrix)

    h, w = matrix.size(-2), matrix.size(-1)
    assert w == 2 * l -1, f'(h, w)= {(h, w)}, l={l}'
    rest = matrix.size()[:-2]

    matrix = matrix.view(-1, h, w)
    b, h, w = matrix.size()

    result = matrix.view(b, -1)
    result = torch.cat([result, torch.zeros(b, l, device=dv)], dim=1)
    assert result.size() == (b, 2 * l * l), f'result.size() {result.size()}'

    result = result.view(b, l, 2*l)
    result = result[:, :, :l]

    result = result.view(*rest, h, l)
    return result

def mask_(matrices, maskval=0.0, mask_diagonal=True):
    """
    Masks out all values in the given batch of matrices where i <= j holds,
    i < j if mask_diagonal is false
    """

    h, w = matrices.size(-2), matrices.size(-1)

    indices = torch.triu_indices(h, w, offset=0 if mask_diagonal else 1)
    matrices[..., indices[0], indices[1]] = maskval

def sample_batch(data, length, batch_size):
    """
    Takes the data (a single sequence of tokens) and slices out a batch of subsequences to provide as input to the model.

    For each input instance, it also slices out the sequence that is shifted one position to the right, to provide as a
    target for the model.
    """

    # Sample the starting indices of the sequences to slice out.
    starts = torch.randint(size=(batch_size,), low=0, high=data.size(0) - length - 1)

    # Slice out the input sequences
    seqs_inputs  = [data[start:start + length] for start in starts]
    # -- the start index is the one we just sampled, and the end is exactly 'lentgh' positions after that.
    seqs_target = [data[start + 1:start + length + 1] for start in starts]
    # -- The target is the same sequence as input, except one character ahead (we are asking the model to predict the
    #    next character at each position)

    # We now have two lists of torch vectors, which we can concatenate into matrices of batch_size-by-length
    inputs = torch.cat([s[None, :] for s in seqs_inputs], dim=0).to(torch.long)
    target = torch.cat([s[None, :] for s in seqs_target], dim=0).to(torch.long)
    # -- Note that we add a singleton dimenson to each vector, s[None.,:], and then concatenate along that dimension.

    return inputs, target

In [13]:
######################################################################
# modules.py

class SelfAttention(nn.Module):
    
    def __init__(self, emb, heads=8, mask=False, kqnorm=False):
        super().__init__()

        assert emb % heads == 0, f'Embedding dimension ({emb}) should be divisible by nr. of heads ({heads})'

        self.emb = emb
        self.heads = heads
        self.mask = mask

        s = emb // heads
        # - We will break the embedding into `heads` chunks and feed each to a different attention head

        self.tokeys    = nn.Linear(emb, emb, bias=False)
        self.toqueries = nn.Linear(emb, emb, bias=False)
        self.tovalues  = nn.Linear(emb, emb, bias=False)

        self.unifyheads = nn.Linear(emb, emb)

        self.kqnorm = kqnorm
        if kqnorm:
            self.kln = nn.LayerNorm([s])
            self.qln = nn.LayerNorm([s])

    def forward(self, x):

        b, t, e = x.size()
        h = self.heads
        assert e == self.emb, f'Input embedding dim ({e}) should match layer embedding dim ({self.emb})'

        s = e // h

        keys    = self.tokeys(x)
        queries = self.toqueries(x)
        values  = self.tovalues(x)

        keys    = keys.view(b, t, h, s)
        queries = queries.view(b, t, h, s)
        values  = values.view(b, t, h, s)

        if self.kqnorm:
            keys = self.kln(keys)
            queries = self.qln(queries)

        # -- We first compute the k/q/v's on the whole embedding vectors, and then split into the different heads.
        #    See the following video for an explanation: https://youtu.be/KmAISyVvE1Y

        # Compute scaled dot-product self-attention

        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, s)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, s)
        values = values.transpose(1, 2).contiguous().view(b * h, t, s)

        queries = queries / (e ** (1/4))
        keys    = keys / (e ** (1/4))
        # - Instead of dividing the dot products by sqrt(e), we scale the keys and values.
        #   This should be more memory efficient

        # - get dot product of queries and keys, and scale
        dot = torch.bmm(queries, keys.transpose(1, 2))

        assert dot.size() == (b*h, t, t)

        if self.mask: # mask out the upper half of the dot matrix, excluding the diagonal
            mask_(dot, maskval=float('-inf'), mask_diagonal=False)

        dot = F.softmax(dot, dim=2)
        # - dot now has row-wise self-attention probabilities

        # apply the self attention to the values
        out = torch.bmm(dot, values).view(b, h, t, s)

        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, s * h)

        return self.unifyheads(out)

In [14]:
class TransformerBlock(nn.Module):

    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, 
                 dropout=0.0, attention_type='default',
                 pos_embedding=None, sa_kwargs={}):
        super().__init__()

        if attention_type == 'default':
            self.attention = SelfAttention(emb, heads=heads, mask=mask, **sa_kwargs)
        elif attention_type == 'alt':
            self.attention = SelfAttentionAlt(emb, heads=heads, mask=mask)
        elif attention_type == 'wide':
            self.attention = SelfAttentionWide(emb, heads=heads, mask=mask)
        elif attention_type == 'gpt2':
            self.attention = SelfAttentionGPT2(emb, heads=heads, mask=mask)
        elif attention_type == 'narrow':
            self.attention = SelfAttentionNarrow(emb, heads=heads, mask=mask)
        elif attention_type == 'relative':
            assert pos_embedding is not None
            self.attention = SelfAttentionRelative(emb, heads=heads, mask=mask, pos_embedding=pos_embedding)
        else:
            raise Exception(f'Self-attention type {type} not recognized.')
            
        self.mask = mask

        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)

        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),
            nn.Linear(ff_hidden_mult * emb, emb)
        )

        self.do = nn.Dropout(dropout)

    def forward(self, x):

        attended = self.attention(x)
        x = self.norm1(attended + x)
        x = self.do(x)
        fedforward = self.ff(x)
        x = self.norm2(fedforward + x)
        x = self.do(x)

        return x

######################################################################
# transformers.py

class GTransformer(nn.Module):
    """
    Transformer for generating text (character by character).
    """

    def __init__(self, emb, heads, depth, seq_length, num_tokens, attention_type='default'):

        super().__init__()

        self.num_tokens = num_tokens
        self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=(seq_length * 2 - 1 if attention_type=='relative' else seq_length))

        tblocks = []
        for i in range(depth):
            tblocks.append(TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=True, 
                                            attention_type=attention_type, pos_embedding=self.pos_embedding))

        self.tblocks = nn.Sequential(*tblocks)
        self.toprobs = nn.Linear(emb, num_tokens)

    def forward(self, x):
        
        tokens = self.token_embedding(x)
        b, t, e = tokens.size()

        positions = self.pos_embedding(torch.arange(t, device=d()))[None, :, :].expand(b, t, e)
        x = tokens + positions

        x = self.tblocks(x)
        x = self.toprobs(x.view(b*t, e)).view(b, t, self.num_tokens)

        return F.log_softmax(x, dim=2)

In [15]:
######################################################################
# generate.py

# NB, the enwik8 data contains tokens from 9 to 240, but we'll round up to the nearest
# power of two.
NUM_TOKENS = 256

def enwik8(path, n_train=int(90e6), n_valid=int(5e6), n_test=int(5e6)):
    """
    Load the enwik8 dataset from the Hutter challenge.

    Adapted from https://github.com/openai/blocksparse/blob/master/examples/transformer/enwik8.py
    :param path:
    :param n_train:
    :param n_valid:
    :param n_test:
    :return:
    """
    with gzip.open(path) if path.endswith('.gz') else open(path) as file:
        # X = np.fromstring(file.read(n_train + n_valid + n_test), dtype=np.uint8)
        X = np.frombuffer(file.read(n_train + n_valid + n_test), dtype=np.uint8)
        trX, vaX, teX = np.split(X, [n_train, n_train + n_valid])
        return torch.from_numpy(trX), torch.from_numpy(vaX), torch.from_numpy(teX)

def sample(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome log-probabilities
    :param temperature: Sampling temperature. 1.0 follows the given distribution,
        0.0 returns the maximum probability element.
    :return: The index of the sampled element.
    """

    if temperature == 0.0:
        return lnprobs.argmax()

    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)

    return cd.sample()

def sample_sequence(model, seed, max_context, length=600, temperature=0.5, verbose=False):
    """
    Sequentially samples a sequence from the model, token by token.

    :param model:
    :param seed: The sequence to start with.
    :param length: The total number of characters to sample.
    :param temperature: The sampling temperature.
    :param verbose: If true, the sampled sequence is also printed as it is sampled.

    :return: The sampled sequence, including the seed.
    """

    sequence = seed.detach().clone()

    if verbose: # Print the seed, surrounded by square brackets
        print('\nGiven this sequence-------\n')  # , end='', flush=True)
        for c in seed:
            print(str(chr(c)), end='', flush=True)
        print('\n\n-------the transformer continued with\n') #, end='', flush=True)

    for _ in range(length):

        # Input is the tail end of the sampled sequence (as many tokens as the model can handle)
        input = sequence[-max_context:]

        # Run the current input through the model
        output = model(input[None, :])

        # Sample the next token from the probabilitys at the last position of the output.
        c = sample(output[0, -1, :], temperature)

        if verbose:
            print(str(chr(max(32, c))), end='', flush=True)

        sequence = torch.cat([sequence, c[None]], dim=0) # Append the sampled token to the sequence

    print()
    return seed

def create_train_test(arg):
    
    data_train, data_val, data_test = enwik8('enwik8.gz')
    data_train, data_test = (torch.cat([data_train, data_val], dim=0), data_test) \
                            if arg.final else (data_train, data_val)

    if False:
        orig_len = len(data_train)
        n = int(orig_len * 0.1)
        data_train = data_train[:n]
        print(f'Reducing size of data_train from {orig_len} to {n} ({n / orig_len})')
        
    # create the model
    model = GTransformer(emb=arg.embedding_size, heads=arg.num_heads,
                         depth=arg.depth, seq_length=arg.context, num_tokens=NUM_TOKENS,
                         attention_type=arg.attention_type)
    if torch.cuda.is_available():
        model.cuda()
    elif torch.backends.mps.is_available():
        model.to('mps')

    opt = torch.optim.Adam(lr=arg.lr, params=model.parameters())

    # Linear learning rate warmup
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lambda i: min(i / (arg.lr_warmup / arg.batch_size), 1.0))

    # Training loop
    # -- We don't loop over the data, instead we sample a batch of random subsequences each time. This is not strictly
    #    better or worse as a training method, it's just a little simpler.
    #
    instances_seen = 0
    for i in tqdm.trange(arg.num_batches):

        opt.zero_grad()

        source, target = sample_batch(data_train, length=arg.context, batch_size=arg.batch_size)
        instances_seen += source.size(0)

        if torch.cuda.is_available():
            source, target = source.cuda(), target.cuda()
        elif torch.backends.mps.is_available():
            source, target = source.to('mps'), target.to('mps')

        output = model(source) # forward pass

        # Compute the loss
        loss = F.nll_loss(output.transpose(2, 1), target, reduction='mean')

        # tbw.add_scalar('transformer/train-loss', float(loss.item()) * util.LOG2E, i * arg.batch_size, instances_seen)
        # tbw.add_scalar('transformer/time-forward', t, instances_seen)

        loss.backward() # backward pass

        # clip gradients
        # -- If the total gradient vector has a length > x, we clip it back down to x.
        if arg.gradient_clipping > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), arg.gradient_clipping)

        opt.step() # stochastic gradient descent step
        sch.step() # update the learning rate

        # Validate every `arg.test_every` steps. First we compute the
        # compression on the validation data (or a subset),
        # then we generate some random text to monitor progress.
        if i != 0 and (i % arg.test_every == 0 or i == arg.num_batches - 1):
            with torch.no_grad():

                ## Sample and print a random sequence

                # Slice a random seed from the test data, and sample a continuation from the model.
                seedfr = random.randint(0, data_test.size(0) - arg.context)
                seed = data_test[seedfr:seedfr + arg.context].to(torch.long)

                if torch.cuda.is_available():
                    seed = seed.cuda()

                sample_sequence(model, seed=seed, max_context=arg.context, verbose=True, length=arg.sample_length)

                ## Compute validation bits per byte

                upto = data_test.size(0) if i == arg.num_batches - 1 else arg.test_subset
                data_sub = data_test[:upto]

                # bits_per_byte = util.compute_compression(model, data_sub, context=arg.context, batch_size=arg.test_batchsize)
                # -- Since we're not computing gradients, we can increase the batch size a little from what we used in
                #    training.

                # print(f'epoch{i}: {bits_per_byte:.4} bits per byte')
                # tbw.add_scalar(f'transformer/eval-loss', bits_per_byte, i * arg.batch_size, instances_seen)
                # -- 0.9 bit per byte is around the state of the art.

In [16]:
class Args:
    def __init__(self, *args, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

args = Args(num_batches=50_000,  # 1_000_000,
            batch_size=32,
            data=None,
            lr=0.0001,
            tb_dir='./runs',
            embedding_size=128,
            num_heads=8,
            context=256,
            depth=12, 
            seed=1, 
            test_every=2000, 
            test_subset=100000, 
            test_batchsize=64, 
            gradient_clipping=1.0, 
            lr_warmup=5000, 
            sample_length=600, 
            attention_type="default",
            final=False)  # "store_true")

start_time = time.time()
create_train_test(args)
elapsed_time = time.time() - start_time
print(f'Training took {elapsed_time/60/60:.2f} hours.')

  4%|███▌                                                                                     | 2000/50000 [03:24<1:22:29,  9.70it/s]


Given this sequence-------

ovisional Government.  Furthermore, it instructed all IRA units to reaffirm their allegiance to the Irish Republic on [[2 April]].

On [[9 April]], the (anti-Treaty) Army Convention reconvened in Dublin. It adopted a new constitution and elected a new 16-m

-------the transformer continued with

ast bof the win sthe in ce tofing the ton thow [[huste e tof coromen a end pacater o and and the of cospermers of ingncerents te tof the the come o the the wit the the ple the a sthe of ate the the cowe the leallll be the the wand the as be ame gee e the the the the and the a the the ain at the bat win the the sean the an the the te a we inoo t the of isthe the the gne are t the the the the the bume tir of the the ofus the the the the be weve trore so the the se the ane te af tre wene see tren te the tof the t thein the it the the the ame the tor the the ase fre se tole the te the the the the 

  4%|███▌                                                                                    | 2001/50000 [03:30<26:05:47,  1.96s/it]

  8%|███████                                                                                  | 4000/50000 [06:55<1:20:09,  9.56it/s]


Given this sequence-------

-like dome.
*HAL was also &quot;featured&quot; in a short commercial by [[Apple Computer]] in the year [[1999]] in which he asks Dave if he likes his Mac more than him (because Macs would not undergo the same potential problems with the &quot;[[Year 2000 p

-------the transformer continued with

rined to contric the streation he the singne a or of the and inted of a comp the contromed to the [[Iman of the commamed, and on the ase on a sell for of clanded of the [[comporition]].  The as the had the of the wo the and to he prodent of the of the spert of so in that of ithe many of the [[artugation]].  The and a a parter of the reather inter the was he and of the a calllequor was in the be and to the sprow.  on the stat contions to a comp onit the the for abot was and as prond of a the ore a const on of a the ongent complicity (shasord of the the ''sperined of the the chirsture of the a c

...

 96%|██████████████████████████████████████████████████████████████████████████████████▌   | 48001/50000 [1:24:08<1:05:05,  1.95s/it]

100%|███████████████████████████████████████████████████████████████████████████████████████▉| 49999/50000 [1:27:31<00:00,  9.95it/s]


Given this sequence-------

 of Germany|German flag]] on the arm; when he took it off, the pale [[khaki]] shirt underneath showed a [[Wehrmacht]] national emblem on the collar and a [[swastika]] armband. A reporter from ''[[The Sun (newspaper)|The Sun]]'' who had been tipped off prev

-------the transformer continued with

ious in [[1945]] to [[1941]] the [[1915]] to 10 showed the [[Sun Bonn]].  The [[1920s]] and [[1944]] and [[1917]], and the [[March Bonnesser Sun Anders Comparison]] in [[1931]], and the ''[[March Bonnesser Bonnesser Bonnesser]]'' and [[1941]] and [[1985]].  It was a comparisoned of the [[Simon Bonnesser Comparison]] and [[Canada Bonnesser Comparison]] and [[Bonnesser Bonnesser]] and [[1950]] and the [[March 1931]] in [[1997]].  For example the [[Persian British Comparison]] and [[1954]] the [[1940s]] of [[1980]].  At the [[National Theology]] is a [[International Division]] in [[1976]].  The [

100%|████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [1:27:38<00:00,  9.51it/s]


Training took 1.46 hours.
